# SQL Project

The coronavirus took the entire world by surprise, changing everyone's daily routine. City dwellers no longer spent their free time outside, going to cafes and malls; more people were home, reading books. That attracted the attention of startups that rushed to develop new apps for book lovers.

You've been given a database of one of the services competing in this market. It contains data on books, publishers, authors, and customer ratings and reviews of books. This information will be used to generate a value proposition for a new product.

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates
import seaborn as sns
import math
import matplotlib.dates as mdates
import datetime 
import math
from sqlalchemy import create_engine

## Download the data

### Connect to the database

In [2]:
db_config = {'user': 'practicum_student',         # username
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### Query engine for data

In [3]:
books = pd.io.sql.read_sql('''SELECT * FROM books''', con = engine)

# books table - convert to datetime
books['publication_date'] = pd.to_datetime(books['publication_date'], format="%Y-%m-%d")

display(books)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


The books table contains information in books such as book id, author id, book title, number of pages in a book, publication date and publisher id. 

In [4]:
authors = pd.io.sql.read_sql('''SELECT * FROM authors''', con = engine)
display(authors)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
...,...,...
631,632,William Strunk Jr./E.B. White
632,633,Zadie Smith
633,634,Zilpha Keatley Snyder
634,635,Zora Neale Hurston


The authors table contains information on authors such as author id and author name

In [5]:
publishers = pd.io.sql.read_sql('''SELECT * FROM publishers''', con = engine)
display(publishers)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
...,...,...
335,336,Workman Publishing Company
336,337,Wyatt Book
337,338,Yale University Press
338,339,Yearling


The publishers table contains data on publishers such as publisher id and publisher name.

In [6]:
ratings = pd.io.sql.read_sql('''SELECT * FROM ratings''', con = engine)
display(ratings)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
...,...,...,...,...
6451,6452,1000,carolrodriguez,4
6452,6453,1000,wendy18,4
6453,6454,1000,jarvispaul,5
6454,6455,1000,zross,2


The ratings table contains information on book ratings such as the rating id, the associated book id, the username of the person who posted the rating and the rating itself.

In [7]:
reviews = pd.io.sql.read_sql('''SELECT * FROM reviews''', con = engine)
display(reviews)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
...,...,...,...,...
2788,2789,999,martinadam,Later hospital turn easy community. Fact same ...
2789,2790,1000,wknight,Change lose answer close pressure. Spend so now.
2790,2791,1000,carolrodriguez,Authority go who television entire hair guy po...
2791,2792,1000,wendy18,Or western offer wonder ask. More hear phone f...


The reviews table contains information on book reviews such as review id, the associated book id, the username of the person who posted the review and the review itself.

## Find the number of books released after January 1, 2000.

In [8]:
query = ''' SELECT COUNT(*) FROM books WHERE publication_date > '2000-01-01' '''

result = pd.io.sql.read_sql(query, con = engine)
print('The number of books released after January 1st, 2000 is: ' + str(result['count'][0]))

The number of books released after January 1st, 2000 is: 819


## Find the number of user reviews and the average rating for each book.

In [9]:
query = ''' SELECT 
                books.title AS title,
                AVG(ratings.rating) as avg_rating,
                COUNT( DISTINCT reviews.review_id) as num_reviews
            FROM books
            INNER JOIN ratings on ratings.book_id = books.book_id
            INNER JOIN reviews on reviews.book_id = books.book_id
            GROUP BY books.book_id
            ORDER BY num_reviews DESC'''

result = pd.io.sql.read_sql(query, con = engine)
display(result.head(10))

,title,avg_rating,num_reviews
0,Twilight (Twilight #1),3.662500,7
1,The Road,3.772727,6
2,The Book Thief,4.264151,6
3,The Glass Castle,4.206897,6
4,Water for Elephants,3.977273,6
5,The Da Vinci Code (Robert Langdon #2),3.830508,6
6,The Catcher in the Rye,3.825581,6
7,The Curious Incident of the Dog in the Night-Time,4.081081,6
8,Outlander (Outlander #1),4.125000,6
9,Harry Potter and the Chamber of Secrets (Harry...,4.287500,6


The table above displays the query results for the number of user reviews and the average ratings for the top 10 books. 

## Identify the publisher that has released the greatest number of books with more than 50 pages.

In [10]:
# by reviewer 20:32 31.12.2020
query = ''' SELECT publishers.publisher as publisher, COUNT( DISTINCT books.book_id) as num_books
            FROM publishers
            INNER JOIN books on books.publisher_id = publishers.publisher_id
            WHERE books.num_pages > 50
            GROUP BY publisher
            ORDER BY num_books DESC'''

result = pd.io.sql.read_sql(query, con = engine)
display(result.head(10))

,publisher,num_books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19
5,Ballantine Books,19
6,Berkley,17
7,Berkley Books,14
8,St. Martin's Press,14
9,William Morrow Paperbacks,13


The table above displays the top 10 publishers who released the greatest number of books with more than 50 pages.

## Identify the author with the highest average book rating: only books with at least 50 ratings.

In [12]:
query = ''' SELECT 
                authors.author as author_name,
                AVG(ratings.rating) as avg_rating
            FROM authors
            INNER JOIN books on books.author_id = authors.author_id
            INNER JOIN ratings on ratings.book_id = books.book_id
            GROUP BY author_name
            HAVING COUNT(ratings.rating) > 50
            ORDER BY avg_rating DESC
                '''

result = pd.io.sql.read_sql(query, con = engine)
display(result.head(10))

,author_name,avg_rating
0,J.K. Rowling/Mary GrandPré,4.288462
1,Agatha Christie,4.283019
2,Markus Zusak/Cao Xuân Việt Khương,4.264151
3,J.R.R. Tolkien,4.240964
4,Roald Dahl/Quentin Blake,4.209677
5,Louisa May Alcott,4.203704
6,Rick Riordan,4.130952
7,Arthur Golden,4.107143
8,Stephen King,4.009434
9,John Grisham,3.971429


The table above displays the authors with the highest average book rating for books only with greater than 50 pages. 

## Find the average number of text reviews among users who rated more than 50 books.

In [13]:
query = ''' SELECT reviews.username, count(reviews.review_id) as avg_num_reviews
            FROM reviews
            WHERE reviews.username in (SELECT ratings.username AS user            
                    FROM ratings
                    GROUP BY ratings.username
                    HAVING COUNT(ratings.rating_id) > 50) 
            GROUP BY reviews.username
            '''

result = pd.io.sql.read_sql(query, con = engine)
display(result.head(10))

,username,avg_num_reviews
0,sfitzgerald,28
1,jennifermiller,25
2,xdavis,18
3,paul88,22
4,martinadam,27
5,richard89,26


The table above shows the average number of text reviews among users who rated more than 50 books. There were only 6 users who rated more than 50 books and using these users, I calculated the average number of text reviews for them. 